# Processamento de Dados

Fonte de Dados: fundsexplorer

In [ ]:
!apt update > /dev/null
!apt install chromium-chromedriver > /dev/null
!pip install selenium beautifulsoup4 pandas > /dev/null



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [ ]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO

In [ ]:
# Configuração do Chrome headless
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)
driver.set_window_size(1920, 1080)  # Garante renderização completa da tela
wait_time = 30

# Acessa o site
driver.get("https://www.fundsexplorer.com.br/ranking")
time.sleep(3)

# Remove overlays e popups
driver.execute_script("""
    let popup = document.querySelector("iframe[aria-label='Popup CTA']");
    if (popup) popup.remove();

    let overlay = document.querySelector("div[id^='hs-overlay-cta-']");
    if (overlay) overlay.remove();

    let modal = document.querySelector("div[id^='hs-interactives-modal-overlay']");
    if (modal) modal.remove();
""")

# Força clique no botão "Colunas Selecionadas" usando JS
driver.execute_script("""
    let btn = document.querySelector("#colunas-ranking__select-button");
    if (btn) btn.click();
""")
time.sleep(2)

# Força clique no checkbox "Selecionar Todos" usando JS no <span>
driver.execute_script("""
    let checkbox = document.querySelector("label[for='colunas-ranking__todos'] span.checkmark");
    if (checkbox) checkbox.click();
""")
time.sleep(2)

# Aguarda a tabela aparecer
time.sleep(3)
html = driver.page_source
driver.quit()

# Parse com BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
table = soup.find("table")

# Converte para DataFrame
df = pd.read_html(StringIO(str(table)))[0]

#print(df.head())


In [ ]:
dados_fund = df.copy()


In [ ]:
def convert_and_divide(df, coluna, divisor):
    if coluna in df.columns:
        df[coluna] = (
            df[coluna]
            .astype(str)
            .str.replace('%', '', regex=False)
            .str.replace('.', '', regex=False)     # remove separador de milhar
            .str.replace(',', '.', regex=False)    # converte decimal
        )
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
        df[coluna] = df[coluna] / divisor
    return df


In [ ]:
dados_fund = convert_and_divide(dados_fund, 'Preço Atual (R$)', 100)
dados_fund = convert_and_divide(dados_fund, 'Último Dividendo', 100)
dados_fund = convert_and_divide(dados_fund, 'Volatilidade', 100)
dados_fund = convert_and_divide(dados_fund, 'P/VP', 1000)
dados_fund = convert_and_divide(dados_fund, 'P/VPA', 100)


In [ ]:
colunas_desejadas = [
    'Fundos', 'Setor', 'Preço Atual (R$)', 'Liquidez Diária (R$)',
    'Último Dividendo', 'Dividend Yield', 'DY (3M) Acumulado',
    'DY (6M) Acumulado', 'DY (12M) Acumulado', 'DY (3M) média',
    'DY (6M) média', 'DY (12M) média', 'DY Ano', 'Variação Preço',
    'Rentab. Acumulada', 'Patrimônio Líquido', 'P/VP',
    'Quant. Ativos', 'Num. Cotistas'
]

df_colunas_filtradas = dados_fund[colunas_desejadas].copy()


In [ ]:
import numpy as np

for col in ['Dividend Yield', 'DY (3M) Acumulado', 'DY (6M) Acumulado', 'DY (12M) Acumulado', 'DY (3M) média', 'DY (6M) média', 'DY (12M) média', 'DY Ano', 'Variação Preço', 'Rentab. Acumulada']:
    if col in df_colunas_filtradas.columns:
        df_colunas_filtradas[col] = df_colunas_filtradas[col].astype(str).str.replace('%', '', regex=False).str.replace(',', '.', regex=False)
        df_colunas_filtradas[col] = pd.to_numeric(df_colunas_filtradas[col], errors='coerce')

for col in ['Liquidez Diária (R$)', 'Patrimônio Líquido']:
    if col in df_colunas_filtradas.columns:
        df_colunas_filtradas[col] = df_colunas_filtradas[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        df_colunas_filtradas[col] = pd.to_numeric(df_colunas_filtradas[col], errors='coerce')

for col in ['Num. Cotistas']:
    if col in df_colunas_filtradas.columns:
        df_colunas_filtradas[col] = df_colunas_filtradas[col].astype(str).str.replace('.', '', regex=False)
        df_colunas_filtradas[col] = pd.to_numeric(df_colunas_filtradas[col], errors='coerce')

dtype_mapping = {
    'Fundos': 'string',
    'Setor': 'category',
    'Preço Atual (R$)': np.float64,
    'Liquidez Diária (R$)': np.float64,
    'Último Dividendo': np.float64,
    'Dividend Yield': np.float64,
    'DY (3M) Acumulado': np.float64,
    'DY (6M) Acumulado': np.float64,
    'DY (12M) Acumulado': np.float64,
    'DY (3M) média': np.float64,
    'DY (6M) média': np.float64,
    'DY (12M) média': np.float64,
    'DY Ano': np.float64,
    'Variação Preço': np.float64,
    'Rentab. Acumulada': np.float64,
    'Patrimônio Líquido': np.float64,
    'P/VP': np.float64,
    'Quant. Ativos': np.int64,
    'Num. Cotistas': np.int64
}

for col, dtype in dtype_mapping.items():
    if col in df_colunas_filtradas.columns:
        if dtype in [np.int64, 'int64'] and df_colunas_filtradas[col].isnull().any():
            df_colunas_filtradas[col] = df_colunas_filtradas[col].astype('Int64')
        else:
            df_colunas_filtradas[col] = df_colunas_filtradas[col].astype(dtype)


In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)


# Classificação dos Fundos


In [ ]:
df_classifc_funds = df_colunas_filtradas.copy()

In [ ]:
df_classifc_funds.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Preço Atual (R$),498.00,92.43,151.77,0.21,10.26,70.06,96.50,995.99
Liquidez Diária (R$),482.00,14801930.34,214462404.88,83.00,25619.18,149917.48,755210.34,4233095626.05
Último Dividendo,529.00,6.87,38.56,0.00,0.13,0.72,1.20,634.23
Dividend Yield,525.00,4.53,23.96,0.00,0.42,0.98,1.26,336.98
DY (3M) Acumulado,525.00,3.95,18.52,0.00,0.00,2.59,3.54,336.98
DY (6M) Acumulado,524.00,10.86,39.54,0.00,2.96,6.06,7.52,665.55
DY (12M) Acumulado,524.00,17.64,53.12,0.00,5.13,11.43,14.42,728.03
DY (3M) média,525.00,1.60,9.81,0.00,0.00,0.87,1.18,185.88
DY (6M) média,524.00,2.45,11.54,0.00,0.53,1.02,1.28,185.88
DY (12M) média,525.00,3.76,34.69,0.00,0.54,1.03,1.24,758.56


In [ ]:
df_classifc_funds = df_classifc_funds[
    (df_classifc_funds['P/VP'] >= 0.80) &
    (df_classifc_funds['Num. Cotistas'] >= 15000) &
    (df_classifc_funds['Patrimônio Líquido'] > 200000000) &
    (df_classifc_funds['Liquidez Diária (R$)'] > 400000) &
    (df_classifc_funds['Dividend Yield'] > 0.65)
]

# Visualização das Tabelas


In [ ]:
df_classifc_funds = df_classifc_funds.sort_values(by='Patrimônio Líquido', ascending=False)
df_classifc_funds

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) média,DY (6M) média,DY (12M) média,DY Ano,Variação Preço,Rentab. Acumulada,Patrimônio Líquido,P/VP,Quant. Ativos,Num. Cotistas
261,KNCR11,Papéis,104.63,10925039.32,1.16,1.14,3.23,6.31,12.44,1.08,1.05,1.04,5.34,-0.44,9.87,7783806997.98,1.03,14,401191
264,KNIP11,Papéis,91.34,5639510.05,1.25,1.39,3.70,7.09,12.58,1.23,1.18,1.05,6.03,1.16,12.84,7394444153.89,0.99,14,72815
519,XPML11,Shoppings,104.60,10193400.55,0.92,0.87,2.72,5.68,11.15,0.91,0.95,0.93,4.68,2.63,20.07,6658222602.66,0.89,15,592434
192,HGLG11,Imóveis Industriais e Logísticos,161.21,4984609.23,1.10,0.69,2.17,4.39,8.79,0.72,0.73,0.73,3.64,6.04,13.54,5498701129.37,0.99,28,501396
268,KNRI11,Misto,146.35,6025388.00,1.00,0.68,2.20,4.54,8.80,0.73,0.76,0.73,3.75,10.40,21.61,4560889697.37,0.91,20,298421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,CYCR11,Indefinido,9.04,665263.23,0.11,1.28,3.76,7.49,14.70,1.25,1.25,1.23,6.30,2.13,15.14,343667848.79,0.96,14,18437
442,SNFF11,Fundo de Fundos,73.10,458213.73,0.72,0.98,3.03,6.07,13.03,1.01,1.01,1.09,5.10,6.23,4.14,335809901.37,0.88,14,28705
334,OUJP11,Papéis,81.75,442648.23,1.05,1.35,3.93,8.39,15.46,1.31,1.40,1.29,7.14,0.05,15.76,323533118.25,0.81,14,25151
441,SNEL11,Fundo de Desenvolvimento,8.58,980190.14,0.10,1.18,3.53,7.12,14.68,1.18,1.19,1.22,5.92,-0.24,7.97,311670932.13,1.06,14,28933


In [ ]:
df_classifc_funds = df_classifc_funds.sort_values(by='Num. Cotistas', ascending=False)
df_classifc_funds

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) média,DY (6M) média,DY (12M) média,DY Ano,Variação Preço,Rentab. Acumulada,Patrimônio Líquido,P/VP,Quant. Ativos,Num. Cotistas
311,MXRF11,Papéis,9.57,10524040.64,0.10,1.09,3.11,6.39,12.60,1.04,1.07,1.05,5.25,2.56,10.03,4111884080.90,1.01,2,1290269
519,XPML11,Shoppings,104.60,10193400.55,0.92,0.87,2.72,5.68,11.15,0.91,0.95,0.93,4.68,2.63,20.07,6658222602.66,0.89,15,592434
192,HGLG11,Imóveis Industriais e Logísticos,161.21,4984609.23,1.10,0.69,2.17,4.39,8.79,0.72,0.73,0.73,3.64,6.04,13.54,5498701129.37,0.99,28,501396
261,KNCR11,Papéis,104.63,10925039.32,1.16,1.14,3.23,6.31,12.44,1.08,1.05,1.04,5.34,-0.44,9.87,7783806997.98,1.03,14,401191
477,VGHF11,Misto,7.75,2577888.55,0.09,1.17,3.64,7.45,14.27,1.21,1.24,1.19,6.21,1.19,12.06,1407310808.76,0.91,14,396499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,JSAF11,Fundo de Fundos,8.03,1313717.73,0.09,1.15,3.69,7.58,14.28,1.23,1.26,1.19,6.38,4.50,10.90,690797148.38,0.89,14,20027
99,CPSH11,Indefinido,9.70,2771812.05,0.10,1.05,3.32,6.97,13.46,1.11,1.16,1.12,5.66,8.04,20.00,945906110.95,0.82,7,19596
115,CYCR11,Indefinido,9.04,665263.23,0.11,1.28,3.76,7.49,14.70,1.25,1.25,1.23,6.30,2.13,15.14,343667848.79,0.96,14,18437
236,ITRI11,Shoppings,82.80,720469.05,0.80,0.99,3.02,6.25,12.32,1.01,1.04,1.03,5.19,7.20,12.77,576776988.88,0.90,14,16993


# Exportação para Google Sheets


In [ ]:
import gspread
from google.colab import auth
from google.auth import default
from gspread_dataframe import set_with_dataframe
from gspread.exceptions import SpreadsheetNotFound

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

try:
    spreadsheet = gc.open('fiis_filtrados')
except SpreadsheetNotFound:
    spreadsheet = gc.create('fiis_filtrados')

worksheet = spreadsheet.get_worksheet(0) or spreadsheet.add_worksheet(title='Sheet1', rows=100, cols=20)

set_with_dataframe(worksheet, df_classifc_funds, include_index=False, include_column_header=True, resize=True)

print("Planilha criada ou atualizada com sucesso.")


Planilha criada ou atualizada com sucesso.
